<a href="https://colab.research.google.com/github/BhagwatiOracle/Generative-AI-using-LangChain/blob/main/RAG%2BMemory_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_groq langchain_huggingface

In [4]:
from google.colab import userdata
groq_api_key=userdata.get('groq_api_key')
LANGCHAIN_API_KEY=userdata.get('LANGCHAIN_API_KEY')
HUGGINGFACE_API_KEY=userdata.get('HUGGINGFACE_HUB_TOKEN')

In [5]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_API_KEY']=LANGCHAIN_API_KEY
os.environ['GROQ_API_KEY']=groq_api_key
os.environ['LANGCHAIN_PROJECT']='AI_AGENT'
os.environ['HUGGINGFACEHUB_API_TOKEN']=HUGGINGFACE_API_KEY

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from langchain_groq import ChatGroq
model=ChatGroq(model='Gemma2-9b-It')

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model=HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader(web_path='https://blog.langchain.dev/what-is-an-agent/')

In [10]:
doc=loader.load()

In [11]:
doc

[Document(metadata={'source': 'https://blog.langchain.dev/what-is-an-agent/', 'title': 'What is an AI agent?', 'description': 'Introducing a new series of musings on AI agents, called "In the Loop".', 'language': 'en'}, page_content='\n\n\nWhat is an AI agent?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhat is an AI agent?\nIntroducing a new series of musings on AI agents, called "In the Loop".\n\nHarrison Chase\n4 min read\nJun 28, 2024\n\n\n\n\n\n‚ÄúWhat is an agent?‚ÄùI get asked this question almost daily. At LangChain, we build tools to help developers build LLM applications, especially those that act as a reasoning engines and interact with external sources of data and computation. This includes systems that are co

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(doc)

In [13]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

In [14]:
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
system_prompt = (
     "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"

)

chat_prompt = ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('human','{input}'),
])

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

In [16]:
from langchain.chains import create_history_aware_retriever

In [17]:
retriever_prompt = (
     "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [18]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',retriever_prompt),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human','{input}'),
    ]

)

In [19]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [20]:
qa_prompt= ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{input}'),
])

In [24]:
# final llm
from langchain.chains.combine_documents import create_stuff_documents_chain
question_answer_chain = create_stuff_documents_chain(model,qa_prompt)

In [25]:
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [27]:
 # chat history

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}

def get_session_history(session_id: str)-> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]


In [28]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key = 'answer',
)

In [29]:
conversational_rag_chain.invoke(
    {'input':'What is Task Decomposition?'},
    config={'configurable':{'session_id':'abc123'}},
)['answer']

'The level of "agenticness" in a system describes how much an LLM controls the application\'s flow. A more agentic system uses the LLM to make decisions about its behavior, potentially including branching logic, cycles, and interactions with external sources. Simple routing systems are considered slightly agentic, while systems that build tools and remember them for future use are highly agentic. \n\n\n'